In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from lxml import etree

In [2]:
main_url = 'https://century21aruba.com/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
    "Accept-Encoding": "*",
    "Connection": "keep-alive"
    }

hyper = ['https://century21aruba.com/en/s/residential/for-sale/single-family-homes',
        'https://century21aruba.com/en/s/residential/for-sale/single-family-homes/hga/2',
         'https://century21aruba.com/en/s/residential/for-sale/single-family-homes/hga/3',
         'https://century21aruba.com/en/s/residential/for-sale/single-family-homes/hga/4',
         'https://century21aruba.com/en/s/residential/for-sale/condos-apartments',
         'https://century21aruba.com/en/s/residential/for-sale/condos-apartments/hga/2',
         'https://century21aruba.com/en/s/residential/for-sale/condos-apartments/hga/3',
         'https://century21aruba.com/en/s/residential/vacant-land',
         'https://century21aruba.com/en/s/residential/vacant-land/hga/2',
         'https://century21aruba.com/en/s/residential/for-rent/single-family-homes',
         'https://century21aruba.com/en/s/residential/for-rent/single-family-homes/hga/2'
         ]

In [4]:
def get_all_url(li):
    all_links = []
    for url in li:
        req = requests.get(url, headers = headers)
        soup = BeautifulSoup(req.content, "html.parser")
        anchors = soup.find('div', {'id': 'search-ajax-contents'})
        links  = anchors.find_all('a', href = True)
        for link in links:
            urls = main_url + link.get('href')
            print(urls)
            all_links.append(urls)
    return all_links
path = get_all_url(hyper)

https://century21aruba.com//en/s/residential/for-sale/single-family-homes/hga/2
https://century21aruba.com//en/s/residential/for-sale/single-family-homes/hga/3
https://century21aruba.com//en/s/residential/for-sale/single-family-homes/hga/4
https://century21aruba.com//en/d/bubali-115-d-single-family-homes-for-sale-bubali-noord-1147981
https://century21aruba.com//en/d/bubali-115-d-home-for-sale-aruba-1147981
https://century21aruba.com//en/d/koyari-23-single-family-homes-for-sale-koyari-noord-1144695
https://century21aruba.com//en/d/koyari-23-home-for-sale-aruba-1144695
https://century21aruba.com//en/d/mesa-vista-8-single-family-homes-for-sale-malmok-noord-1148794
https://century21aruba.com//en/d/mesa-vista-8-home-for-sale-aruba-1148794
https://century21aruba.com//en/d/tuscany-villa-14-single-family-homes-for-sale-malmok-noord-1144690
https://century21aruba.com//en/d/tuscany-villa-14-home-for-sale-aruba-1144690
https://century21aruba.com//en/d/esmeralda-82-single-family-homes-for-sale-wes

In [5]:
path2 = []
 
for i in path:
    if i not in path2:
        path2.append(i)
path3 = list(reversed(path2))
path3

['https://century21aruba.com//en/d/koyari-home-for-rent-aruba-1149389',
 'https://century21aruba.com//en/d/koyari-single-family-homes-for-rent-koyari-noord-1149389',
 'https://century21aruba.com//en/d/matadera-home-for-rent-aruba-1149410',
 'https://century21aruba.com//en/d/matadera-single-family-homes-for-rent-matadera-noord-1149410',
 'https://century21aruba.com//en/d/fuerte-home-for-rent-aruba-1148990',
 'https://century21aruba.com//en/d/fuerte-single-family-homes-for-rent-paradera-paradera-1148990',
 'https://century21aruba.com//en/d/fuerte-61-home-for-rent-aruba-1149884',
 'https://century21aruba.com//en/d/fuerte-61-single-family-homes-for-rent-paradera-paradera-1149884',
 'https://century21aruba.com//en/d/kudawecha-home-for-rent-aruba-1148947',
 'https://century21aruba.com//en/d/kudawecha-single-family-homes-for-rent-kudawecha-noord-1148947',
 'https://century21aruba.com//en/s/residential/for-rent/single-family-homes/',
 'https://century21aruba.com//en/d/paradera-115-t-home-for-r

In [8]:
def get_details(urls):
    property_list = []
    for link in urls:
        req = requests.get(link, headers = headers)
        soup = BeautifulSoup(req.content, 'html.parser')
        dom = etree.HTML(str(soup))
        try:
            name = soup.find('span', class_ = 'd-block h2 mb-0 text-sm-left autofit-text font-weight-bold').text.strip()
            try:
                property_status = soup.find('b', class_ = 'text-shadow px-3').text.strip()
                price= soup.find('span', class_ = 'h3 m-0 border-bottom').text.split('/')[0]
            except AttributeError:
                property_status = None
                price = soup.find('span', class_ = 'd-block price-sold').text
            #price= soup.find('span', class_ = 'h3 m-0 border-bottom').text.split('/')[0]
            pool = soup.find(lambda t: t.name == 'li' and 'View' in t.text).text.split(" ")[1]
            location = soup.find(lambda t: t.name == 'li' and 'Location' in t.text).text.split(" ")[1]            
            rooms = soup.find(lambda t: t.name == 'li' and 'Beds' in t.text).text.split(" ")[1]
            bathroom = soup.find(lambda t: t.name == 'li' and 'Baths' in t.text).text.split(" ")[1]
        except AttributeError:
            pass
       
            
        prop = {
            'name': name,
            'location': location,
            'property': property_status,
            'price': price,
            'bedrooms': rooms,
            'bathrooms' : bathroom,
            'Pool':pool,
            'link': link
        }
        property_list.append(prop)
        print('Saving:', prop['name'])
        
    df = pd.DataFrame(property_list)
    df = df.drop_duplicates()
    print(df.head(20))
    df.to_csv('aruba_houses.csv', header=True, index=False)

In [9]:
get_details(path3)

Saving: Koyari
Saving: Koyari
Saving: Matadera
Saving: Matadera
Saving: Fuerte
Saving: Fuerte
Saving: Fuerte 61
Saving: Fuerte 61
Saving: Kudawecha
Saving: Kudawecha
Saving: Kudawecha
Saving: Paradera 115 - T
Saving: Paradera 115 - T
Saving: Monte Verde 21
Saving: Monte Verde 21
Saving: Macuarima
Saving: Macuarima
Saving: Nuñe
Saving: Nuñe
Saving: Quillstraat 36
Saving: Quillstraat 36
Saving: Washington
Saving: Washington
Saving: Calabas
Saving: Calabas
Saving: Calabas
Saving: San Barbola 279
Saving: Washington Gardens
Saving: Washington Gardens
Saving: Paradera 225
Saving: Paradera 225
Saving: Paradera 225
Saving: Beatrixstraat
Saving: Mesavista Tierra Del Sol Villa
Saving: Mesavista Tierra Del Sol Villa
Saving: Mesavista Tierra Del Sol Villa
Saving: Modanza Homes & Gardens Property Land
Saving: Modanza Homes & Gardens Property Land
Saving: Modanza Homes & Gardens Property Land
Saving: Tamarijn Homes And Gardens
Saving: Tamarijn Homes And Gardens
Saving: Napa Valley Residences Propert

In [25]:
data = pd.read_csv('aruba_houses.csv')

In [26]:
data["name"] = data['name'].drop_duplicates()
data.head()

,name,location,property,price,bedrooms,bathrooms,Pool,link
0,Koyari,Koyari,New Listing,"US$ 1,039",2,2,Other,https://century21aruba.com//en/d/koyari-home-f...
1,NaN,Koyari,New Listing,"US$ 1,039",2,2,Other,https://century21aruba.com//en/d/koyari-single...
2,Matadera,Matadera,Under Contract,"US$ 1,124",2,1,Garden,https://century21aruba.com//en/d/matadera-home...
3,NaN,Matadera,Under Contract,"US$ 1,124",2,1,Garden,https://century21aruba.com//en/d/matadera-sing...
4,Fuerte,Paradera,NaN,RENTED,3,2,Garden,https://century21aruba.com//en/d/fuerte-home-f...


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       73 non-null     object
 1   location   217 non-null    object
 2   property   75 non-null     object
 3   price      217 non-null    object
 4   bedrooms   217 non-null    int64 
 5   bathrooms  217 non-null    object
 6   Pool       217 non-null    object
 7   link       217 non-null    object
dtypes: int64(1), object(7)
memory usage: 13.7+ KB


In [28]:
data = data.dropna(subset = ['name'])
data

,name,location,property,price,bedrooms,bathrooms,Pool,link
0,Koyari,Koyari,New Listing,"US$ 1,039",2,2,Other,https://century21aruba.com//en/d/koyari-home-f...
2,Matadera,Matadera,Under Contract,"US$ 1,124",2,1,Garden,https://century21aruba.com//en/d/matadera-home...
4,Fuerte,Paradera,NaN,RENTED,3,2,Garden,https://century21aruba.com//en/d/fuerte-home-f...
6,Fuerte 61,Paradera,Under Contract,"US$ 1,320",3,2,Garden,https://century21aruba.com//en/d/fuerte-61-hom...
8,Kudawecha,Kudawecha,NaN,RENTED,3,2,Garden,https://century21aruba.com//en/d/kudawecha-hom...
...,...,...,...,...,...,...,...,...
204,Esmeralda 82,Westpunt,NaN,SOLD,5,4,Inland,https://century21aruba.com//en/d/esmeralda-82-...
206,Tuscany Villa 14,Westpunt,NaN,SOLD,5,4,Inland,https://century21aruba.com//en/d/tuscany-villa...
208,Mesa Vista 8,Westpunt,NaN,SOLD,5,4,Inland,https://century21aruba.com//en/d/mesa-vista-8-...
210,Koyari 23,Koyari,NaN,SOLD,4,3,Pool,https://century21aruba.com//en/d/koyari-23-hom...


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 212
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       73 non-null     object
 1   location   73 non-null     object
 2   property   35 non-null     object
 3   price      73 non-null     object
 4   bedrooms   73 non-null     int64 
 5   bathrooms  73 non-null     object
 6   Pool       73 non-null     object
 7   link       73 non-null     object
dtypes: int64(1), object(7)
memory usage: 5.1+ KB


In [30]:
data.to_csv('aruba2_houses.csv', header = True, index = False)